<a href="https://colab.research.google.com/github/Im-jDiOt/DeepLearning-Term-Proj/blob/feature-resnet/6%EC%B0%A8%EC%8B%A4%ED%96%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import timm  # timm 라이브러리 사용
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt

# GPU 사용 가능 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'사용 디바이스: {device}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')

# --- 1. 데이터 로드 및 하이퍼파라미터 (캐글 경로 수정) ---

# [수정] 캐글 데이터셋 기본 경로
base_dir = r'/kaggle/input/state-farm-distracted-driver-detection'

# [수정] driver_imgs_list.csv 파일 경로
driver_csv_path = os.path.join(base_dir, 'driver_imgs_list.csv')

# [수정] 'imgs' 폴더 내의 train/test 경로
train_dir = os.path.join(base_dir, 'imgs', 'train')
test_dir = os.path.join(base_dir, 'imgs', 'test')

img_size = 224
batch_size = 32
num_classes = 10
num_epochs = 40
learning_rate = 0.001

# [수정] 캐글은 num_workers=2 또는 4가 더 빠릅니다.
num_workers = 2

print(f"Train directory: {train_dir}")
print(f"Test directory: {test_dir}")
print(f"이미지 크기: {img_size}x{img_size}")
print(f"배치 크기: {batch_size}")

driver_df = pd.read_csv(driver_csv_path)
print(f"고유 운전자 수: {driver_df['subject'].nunique()}명")

# --- 2. 5-Fold 교차 검증 (운전자 기준 분할) ---
all_drivers = sorted(driver_df['subject'].unique())
n_folds = 5
kfold = KFold(n_splits=n_folds, shuffle=True, random_state=42)

fold_splits = []
print("K-Fold 운전자 분할:")
for fold_idx, (train_indices, val_indices) in enumerate(kfold.split(all_drivers)):
    train_drivers = [all_drivers[i] for i in train_indices]
    val_drivers = [all_drivers[i] for i in val_indices]

    fold_splits.append({
        'fold': fold_idx+1,
        'train_drivers': train_drivers,
        'val_drivers': val_drivers
    })

    # (로그 출력은 간결하게)
    train_imgs = driver_df[driver_df['subject'].isin(train_drivers)]
    val_imgs = driver_df[driver_df['subject'].isin(val_drivers)]
    print(f"  Fold {fold_idx+1}: 학습 {len(train_imgs)}개 | 검증 {len(val_imgs)}개")


# --- 3. 커스텀 데이터셋 (DriverDataset) [블랙리스트 기능 추가] ---

class DriverDataset(Dataset):
    """운전자 행동 데이터셋 (블랙리스트 기능 추가)"""

    # [수정] blacklist_files 파라미터 추가
    def __init__(self, data_dir, driver_df, driver_list, transform=None, is_test=False,
                 blacklist_files=None):

        self.data_dir = data_dir
        self.transform = transform
        self.is_test = is_test
        self.images = []
        self.labels = []

        # [수정] 블랙리스트를 set으로 변경 (검색 속도 향상)
        self.blacklist = set(blacklist_files) if blacklist_files else set()

        if self.blacklist and not is_test:
            print(f"   [Dataset] {len(self.blacklist)}개의 학습 이미지를 제외합니다.")

        if is_test:
            test_images_dir = data_dir
            try:
                for img_name in os.listdir(test_images_dir):
                    if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                        # (테스트 이미지는 블랙리스트 적용 안 함)
                        self.images.append(os.path.join(test_images_dir, img_name))
            except Exception as e:
                print(f"테스트 디렉토리 읽기 오류: {e}")
        else: #is_train
            driver_subset = driver_df[driver_df['subject'].isin(driver_list)]

            for _, row in driver_subset.iterrows():
                class_name = row['classname']
                img_name = row['img']

                # [수정] 블랙리스트에 파일명이 있으면 스킵
                if img_name in self.blacklist:
                    continue

                img_path = os.path.join(data_dir, class_name, img_name)
                self.images.append(img_path)
                class_idx = int(class_name[1:])
                self.labels.append(class_idx)

        print(f"{'테스트' if is_test else '운전자 ' + str(len(driver_list))+'명'}, 데이터 {len(self.images)}개 이미지 로드")

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"이미지 로드 오류: {img_path}, {e}")
            return None, None

        if self.transform:
            image = self.transform(image)

        if self.is_test:
            return image, os.path.basename(img_path)
        else:
            label = self.labels[idx]
            return image, label

# --- 4. 데이터 전처리 (Transforms) ---
team_transform_train = transforms.Compose([
  transforms.Resize((img_size, img_size)),
  transforms.RandomRotation(degrees=15),
  # [추가] 좌우 반전은 c1/c3, c2/c4를 헷갈리게 할 수 있어 제외 (또는 라벨 스위칭 필요)
  # transforms.RandomHorizontalFlip(),
  transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.05),
  transforms.ToTensor(),
  transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
])

team_transform_eval = transforms.Compose([
  transforms.Resize((img_size, img_size)),
  transforms.ToTensor(),
  transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
])

# --- 5. 학습/검증 헬퍼 함수 (기존과 동일) ---
def train_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    for inputs, labels in tqdm(train_loader, desc="Training"):
        if inputs is None or labels is None:
            continue

        # (collate_fn에서 빈 배치가 올 수 있음)
        if inputs.nelement() == 0:
            continue

        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct_predictions += torch.sum(preds == labels.data)
        total_samples += inputs.size(0)

    if total_samples == 0:
        return 0.0, 0.0

    epoch_loss = running_loss / total_samples
    epoch_acc = (correct_predictions.double() / total_samples) * 100
    return epoch_loss, epoch_acc.item()

def validate(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():
        for inputs, labels in tqdm(val_loader, desc="Validating"):
            if inputs is None or labels is None:
                continue

            if inputs.nelement() == 0:
                continue

            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct_predictions += torch.sum(preds == labels.data)
            total_samples += inputs.size(0)

    if total_samples == 0:
        return 0.0, 0.0

    epoch_loss = running_loss / total_samples
    epoch_acc = (correct_predictions.double() / total_samples) * 100
    return epoch_loss, epoch_acc.item()


# --- 6. 핵심 훈련 함수 (train_fold) [블랙리스트 파라미터 추가] ---
def train_fold(fold_idx, train_drivers, val_drivers, blacklist_files): # [수정]
    """한 폴드 학습 (2단계 미세 조정 적용)"""

    print(f"\n========== Fold {fold_idx}/{n_folds} ==========")

    # --- 데이터셋 및 로더 준비 ([수정] blacklist_files 전달) ---
    train_dataset = DriverDataset(
        train_dir, driver_df, train_drivers,
        transform=team_transform_train, is_test=False,
        blacklist_files=blacklist_files # [수정]
    )
    val_dataset = DriverDataset(
        train_dir, driver_df, val_drivers,
        transform=team_transform_eval, is_test=False,
        blacklist_files=blacklist_files # [수정]
    )

    def collate_fn(batch):
        batch = list(filter(lambda x: x[0] is not None, batch))
        if not batch: return torch.Tensor(), torch.Tensor()
        return torch.utils.data.dataloader.default_collate(batch)

    train_loader = DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True,
        num_workers=num_workers, collate_fn=collate_fn
    )
    val_loader = DataLoader(
        val_dataset, batch_size=batch_size, shuffle=False,
        num_workers=num_workers, collate_fn=collate_fn
    )

    print(f"학습 배치 수: {len(train_loader)}")
    print(f"검증 배치 수: {len(val_loader)}")

    # --- 모델 로드 ---
    model = timm.create_model(
        'resnet50',
        pretrained=True,
        num_classes = num_classes
    )
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()

    # ==========================================================
    # 1단계: 분류기(Head)만 학습 (특징 추출기는 동결)
    # ==========================================================
    print("\n--- 1단계: 분류기(Head) 학습 시작 ---")

    for param in model.parameters():
        param.requires_grad = False
    for param in model.fc.parameters():
        param.requires_grad = True

    optimizer_head = optim.Adam(model.fc.parameters(), lr=learning_rate, weight_decay=1e-4)
    scheduler_head = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer_head, mode='min', factor=0.5, patience=5
    )
    num_head_epochs = 10

    for epoch in range(num_head_epochs):
        print(f'\n[Head Training] Epoch {epoch+1}/{num_head_epochs}')
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer_head, device)
        val_loss, val_acc = validate(model, val_loader, criterion, device)
        scheduler_head.step(val_loss)
        print(f'  > Head Epoch {epoch+1}: Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%')

    # ==========================================================
    # 2단계: 전체 모델 미세 조정 (전체 동결 해제)
    # ==========================================================
    print("\n--- 2단계: 전체 모델 미세 조정 시작 ---")

    for param in model.parameters():
        param.requires_grad = True

    fine_tune_lr = learning_rate / 10  # (0.0001)
    optimizer = optim.Adam(model.parameters(), lr=fine_tune_lr, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=7
    )
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
    best_val_loss = float('inf')
    best_model_path = f'best_resnet50_fold{fold_idx}.pth'

    remaining_epochs = num_epochs - num_head_epochs

    for epoch in range(remaining_epochs):
        current_epoch = epoch + 1 + num_head_epochs
        print(f'\n[Fine-Tuning] Epoch {current_epoch}/{num_epochs}')

        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
        val_loss, val_acc = validate(model, val_loader, criterion, device)

        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)

        print(f'\n📊 Epoch {current_epoch} 결과:')
        print(f'  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%')
        print(f'  Val Loss:   {val_loss:.4f} | Val Acc:   {val_acc:.2f}%')

        scheduler.step(val_loss)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save({
                'fold': fold_idx,
                'epoch': current_epoch - 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_loss': val_loss,
                'val_acc': val_acc,
            }, best_model_path)
            print(f'  ✓ 최고 성능 모델 저장! (Val Loss: {val_loss:.4f})')

    print(f"\n✓ Fold {fold_idx} 학습 완료! 최저 검증 손실: {best_val_loss:.4f}")

    return {
        'fold': fold_idx,
        'best_val_loss': best_val_loss,
        'best_val_acc': max(history['val_acc']) if history['val_acc'] else 0.0,
        'model_path': best_model_path
    }

# --- 7. 메인 실행 [핵심 수정: 5-Fold 전체 실행] ---

# [수정] 1단계 분석에서 찾은 '나쁜 라벨' 50개 리스트
BLACKLIST_FILES = [
    # Row 1 (True: c7, Pred: c0)
    'img_88198.jpg', 'img_5883.jpg', 'img_21045.jpg', 'img_58599.jpg', 'img_65624.jpg',
    # Row 2 (True: c2, Pred: c5)
    'img_101662.jpg', 'img_57065.jpg', 'img_55392.jpg', 'img_101830.jpg', 'img_57106.jpg',
    # Row 3 (True: c0, Pred: c2)
    'img_77353.jpg', 'img_77264.jpg', 'img_77364.jpg', 'img_77387.jpg', 'img_77322.jpg',
    # Row 4 (True: c0, Pred: c2)
    'img_100361.jpg', 'img_100130.jpg', 'img_100122.jpg', 'img_100150.jpg', 'img_100170.jpg',
    # Row 5 (True: c0, Pred: c3)
    'img_22240.jpg', 'img_22288.jpg', 'img_22263.jpg', 'img_22285.jpg', 'img_22267.jpg',
    # Row 6 (True: c0, Pred: c4) - 명백한 라벨 오류
    'img_8576.jpg', 'img_8494.jpg', 'img_8556.jpg', 'img_8524.jpg', 'img_8471.jpg',
    # Row 7 (True: c0, Pred: c4) - 명백한 라벨 오류
    'img_41639.jpg', 'img_41620.jpg', 'img_41648.jpg', 'img_41656.jpg', 'img_41673.jpg',
    # Row 8 (True: c0, Pred: c4) - 명백한 라벨 오류
    'img_87271.jpg', 'img_87211.jpg', 'img_87258.jpg', 'img_87192.jpg', 'img_87220.jpg',
    # Row 9 (True: c0, Pred: c4) - 명백한 라벨 오류
    'img_69192.jpg', 'img_69187.jpg', 'img_69171.jpg', 'img_69176.jpg', 'img_69168.jpg',
    # Row 10 (True: c0, Pred: c4) - 명백한 라벨 오류
    'img_8003.jpg', 'img_8022.jpg', 'img_8032.jpg', 'img_8055.jpg', 'img_8065.jpg'
]
print(f"데이터 클렌징: {len(BLACKLIST_FILES)}개의 잘못된 라벨 이미지를 훈련에서 제외합니다.")

all_fold_results = []
try:
    # [수정] fold_splits[0] 대신 5개 Fold 전체를 루프로 실행
    for fold_info in fold_splits:
        fold_idx = fold_info['fold']
        train_drivers = fold_info['train_drivers']
        val_drivers = fold_info['val_drivers']

        # [수정] train_fold에 blacklist 전달
        fold_result = train_fold(fold_idx, train_drivers, val_drivers, BLACKLIST_FILES)
        all_fold_results.append(fold_result)

        torch.cuda.empty_cache()

    print("\n" + "=" * 70)
    print("✅ 5-Fold 전체 학습 완료!")
    print("=" * 70)

    total_val_loss = 0
    total_val_acc = 0
    for result in all_fold_results:
        print(f"  Fold {result['fold']}: 최저 Val Loss: {result['best_val_loss']:.4f} | 최고 Val Acc: {result['best_val_acc']:.2f}%")
        total_val_loss += result['best_val_loss']
        total_val_acc += result['best_val_acc']

    print(f"\n  > 5-Fold 평균 최저 Val Loss: {total_val_loss / n_folds:.4f}")
    print(f"  > 5-Fold 평균 최고 Val Acc: {total_val_acc / n_folds:.2f}%")

except Exception as e:
    print(f"\n🚨 훈련 중 오류 발생: {e}")
    import traceback
    traceback.print_exc()


# --- 8. 5-Fold 앙상블 예측 및 제출 [핵심 수정] ---
if not all_fold_results:
    print("🚨 오류: 학습된 폴드 결과가 없어 예측을 생략합니다.")
else:
    print("\n" + "=" * 70)
    print(f"🔮 5-Fold 앙상블 예측 시작")
    print("=" * 70)

    # --- 8-1. 테스트 데이터셋 로드 (한 번만) ---
    test_dataset = DriverDataset(
        test_dir, driver_df, [],
        transform=team_transform_eval, is_test=True
    )

    def collate_fn_test(batch):
        batch = list(filter(lambda x: x[0] is not None, batch))
        if not batch: return torch.Tensor(), []
        images, filenames = zip(*batch)
        return torch.stack(images), list(filenames)

    test_loader = DataLoader(
        test_dataset, batch_size=batch_size, shuffle=False,
        num_workers=num_workers, collate_fn=collate_fn_test
    )
    print(f"테스트 샘플: {len(test_dataset)}개")

    # --- 8-2. 5개 모델로 각각 예측 수행 ---
    all_folds_predictions = []

    for result in all_fold_results:
        fold_idx = result['fold']
        model_path = result['model_path']

        print(f"\n  Fold {fold_idx} 예측 중 (모델: {model_path})...")

        # 모델 뼈대 생성 및 가중치 로드
        model = timm.create_model('resnet50', pretrained=False, num_classes=num_classes)
        try:
            checkpoint = torch.load(model_path, map_location=device)
            model.load_state_dict(checkpoint['model_state_dict'])
        except Exception as e:
            print(f"🚨 Fold {fold_idx} 모델 로드 실패: {e}")
            continue # 이 폴드는 건너뛰기

        model = model.to(device)
        model.eval()

        fold_predictions = []
        img_names_this_fold = [] # 이미지 이름은 첫 번째 폴드에서만 가져옴

        with torch.no_grad():
            for images, filenames in tqdm(test_loader, desc=f'Fold {fold_idx} 예측'):
                if images.nelement() == 0: continue

                images = images.to(device)
                outputs = model(images)
                probs = torch.softmax(outputs, dim=1)

                fold_predictions.append(probs.cpu().numpy())

                # [수정] 이미지 이름은 첫 번째 폴드에서만 저장
                if fold_idx == 1:
                    img_names_this_fold.extend(filenames)

        all_folds_predictions.append(np.vstack(fold_predictions))

        # [수정] 첫 번째 폴드에서만 이미지 이름 리스트를 확정
        if fold_idx == 1:
            final_img_names = img_names_this_fold

        torch.cuda.empty_cache()

    # --- 8-3. 예측 결과 앙상블 (평균) ---
    if not all_folds_predictions:
        print("🚨 오류: 유효한 예측 결과가 없습니다.")
    else:
        print(f"\n✓ {len(all_folds_predictions)}개 Fold 예측 완료. 결과 앙상블(평균) 중...")

        # (N_folds, N_images, N_classes) 형태의 배열
        all_folds_predictions_np = np.array(all_folds_predictions)

        # [핵심] 0번 축(Fold 축)을 기준으로 평균 계산
        final_predictions_avg = np.mean(all_folds_predictions_np, axis=0)

        print(f"✓ 앙상블 완료: {final_predictions_avg.shape}")

        # --- 8-4. Submission 파일 생성 ---
        class_cols = [f'c{i}' for i in range(num_classes)]
        submission_data = {'img': final_img_names}
        for i, col in enumerate(class_cols):
            submission_data[col] = final_predictions_avg[:, i]

        submission = pd.DataFrame(submission_data)

        # [수정] 파일 이름 변경
        submission_file = 'resnet50_5fold_ensemble_submission.csv'
        submission.to_csv(submission_file, index=False)

        print("\n" + "=" * 70)
        print(f"✅ 5-Fold 앙상블 Submission 파일 생성 완료: {submission_file}")
        print(f"✅ 총 {len(submission)}개 이미지 예측")
        print("=" * 70)
        print("\n📋 Submission 샘플:")
        print(submission.head())

# --- 9. 오분류 분석 [제거] ---
# (오분류 분석(1단계)은 이미 완료되었고,
#  블랙리스트(2단계)는 7번 섹션에 적용되었습니다.)
# (새로 훈련된 모델로 다시 분석하고 싶다면 이전 9번 코드를 여기에 붙여넣으면 됩니다.)

사용 디바이스: cuda
GPU: Tesla T4
Train directory: /kaggle/input/state-farm-distracted-driver-detection/imgs/train
Test directory: /kaggle/input/state-farm-distracted-driver-detection/imgs/test
이미지 크기: 224x224
배치 크기: 32


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/state-farm-distracted-driver-detection/driver_imgs_list.csv'